In [2]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb
from sklearn.metrics import f1_score
# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')
# load feats
y_preds_all = []
y_trues_all = []
y_test_preds_all = []
for feat in sorted(glob.glob('../feature/bagging/*')):
    print('file path',feat)
    a,b,c = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    y_preds_all.append(a)
    y_trues_all.append(b)
    y_test_preds_all.append(c)
# load y
train = pd.read_csv("../input/new_data/train_set.csv")
# list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
# train_y = train[list_classes].values.astype('int')
# print(train_x.shape)

file path ../feature/bagging/rnn_model_bagging1.pkl
(56590, 19) (56590, 19)
file path ../feature/bagging/rnn_model_bagging2.pkl
(56590, 19) (56590, 19)
file path ../feature/bagging/rnn_model_bagging3.pkl
(56590, 19) (56590, 19)


In [3]:
np.corrcoef(y_preds_all[0][:,0], y_preds_all[1][:,0])

array([[1.        , 0.93459933],
       [0.93459933, 1.        ]])

In [43]:
# y = y_trues_all[0]
y[:,5].shape

(56590,)

In [5]:
np.array(y_preds_all)

array([[[1.8630869e-08, 2.9356926e-07, 2.1145908e-07, ...,
         3.4839523e-08, 1.1018854e-04, 9.9985576e-01],
        [2.4553029e-02, 5.2418782e-06, 2.2986982e-04, ...,
         4.5884890e-04, 2.3034238e-05, 2.9243096e-05],
        [3.3866694e-08, 1.0681297e-11, 9.9990904e-01, ...,
         7.8480855e-10, 1.1327355e-07, 8.7279702e-08],
        ...,
        [4.5157832e-04, 4.2930693e-05, 5.5644205e-03, ...,
         3.3414379e-02, 1.3150198e-03, 1.0587270e-02],
        [8.4539055e-04, 1.5751001e-03, 1.5836615e-02, ...,
         7.9410546e-04, 5.4711738e-04, 1.3594753e-01],
        [1.3048108e-01, 3.6731706e-04, 7.2827017e-05, ...,
         6.3033903e-04, 5.6701113e-04, 1.5297865e-03]],

       [[1.8210084e-06, 7.8562034e-06, 1.9555974e-04, ...,
         1.2859799e-05, 1.1062146e-04, 9.9814069e-01],
        [1.7167586e-01, 1.5972940e-04, 1.5574525e-03, ...,
         1.0878749e-02, 1.4222388e-03, 1.3451822e-03],
        [1.5446903e-07, 3.6922784e-10, 9.9768639e-01, ...,
         5.399

In [6]:
from scipy.stats import gmean, hmean

preds_mean = gmean(np.array(y_preds_all) , axis=0)
# print(roc_auc_score(y_trues_all[0], preds_mean))
test_preds_mean = gmean(np.array(y_test_preds_all), axis=0)

In [7]:
preds_mean

array([[1.2161289e-06, 3.9266970e-06, 1.4805765e-05, ..., 2.4074050e-06,
        2.1898779e-04, 9.9829900e-01],
       [4.7133941e-02, 2.8934423e-05, 4.5251148e-04, ..., 2.4724260e-03,
        1.5939391e-04, 3.4292438e-04],
       [9.2532900e-08, 1.1125236e-10, 9.9903888e-01, ..., 2.3531661e-09,
        6.7025752e-07, 3.7445923e-07],
       ...,
       [4.7558622e-04, 7.0500128e-06, 6.0218368e-05, ..., 5.0377231e-03,
        2.0820987e-04, 8.1735931e-04],
       [6.8229716e-04, 6.1266182e-04, 1.0815197e-02, ..., 4.7289432e-04,
        1.8617201e-04, 1.9766283e-01],
       [8.2947217e-02, 5.5450371e-05, 4.7187077e-05, ..., 4.1734509e-04,
        5.2362605e-04, 2.5443881e-04]], dtype=float32)

In [12]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stemmer.stem("mother")

'mother'

In [13]:
preds_stacked = np.concatenate(np.array(y_preds_all), axis=1)
test_preds_stacked = np.concatenate(np.array(y_test_preds_all), axis=1)

In [44]:
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
cvlist = list(StratifiedKFold(10,random_state=1).split(preds_stacked, y[:,5]))
y_preds2 = np.zeros((preds_stacked.shape[0], 19))

y_test_preds2 = np.zeros((test_preds_stacked.shape[0], 19))

In [46]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score
for i, col in enumerate(range(19)):
    y_tmp = y[:, i]
    model = lgb.LGBMClassifier(n_estimators=100, num_leaves=8, learning_rate=0.1, min_child_samples=500,
                               subsample=0.9, colsample_bytree=0.8, reg_lambda=1.0, class_weight='balanced')

    y_preds2[:, i] = cross_val_predict(model, preds_stacked, y_tmp, cv=cvlist, n_jobs=1, method='predict_proba')[:,1]

    y_test_preds2[:, i] = model.fit(preds_stacked, y_tmp).predict_proba(test_preds_stacked)[:,1]
    print("Score for class {} is {}".format(col, roc_auc_score(y_tmp, y_preds2[:, i])))
print("Over auc score", roc_auc_score(y, y_preds2))

Score for class 0 is 0.97282088051057
Score for class 1 is 0.9881714882493464
Score for class 2 is 0.9901577025569563
Score for class 3 is 0.9920279197003412
Score for class 4 is 0.9853334765061417
Score for class 5 is 0.992407591520911
Score for class 6 is 0.9769423843338143
Score for class 7 is 0.9792017033469735
Score for class 8 is 0.9953281842417061
Score for class 9 is 0.9722154430043937
Score for class 10 is 0.9769806318659203
Score for class 11 is 0.9675512389958606
Score for class 12 is 0.9822447845567661
Score for class 13 is 0.9889765594246825
Score for class 14 is 0.9928519344637105
Score for class 15 is 0.9941799784902019
Score for class 16 is 0.9885119118485288
Score for class 17 is 0.9903284240940756
Score for class 18 is 0.9767356713761584
Over auc score 0.9843667320572137


In [48]:
y_test_preds2.shape

(102277, 19)

In [50]:
test_dir = '../input/new_data/test_set.csv'
test = pd.read_csv(test_dir)
test_id = test[["id"]].copy()

In [51]:
preds=np.argmax(y_test_preds2,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["class"]
test_pred["class"]=(test_pred["class"]+1).astype(int)
print(test_pred.shape)
print(test_id.shape)
test_pred["id"]=list(test_id["id"])
test_pred[["id","class"]].to_csv('../output/bagging1.csv',index=None)

(102277, 1)
(102277, 1)


In [53]:
y_test_preds2

array([[0.04303668, 0.01117231, 0.0170423 , ..., 0.00496908, 0.00838028,
        0.00225438],
       [0.06548195, 0.0014784 , 0.00338679, ..., 0.00075571, 0.00224275,
        0.00674224],
       [0.38615825, 0.03139946, 0.74362352, ..., 0.05014074, 0.05240256,
        0.13416665],
       ...,
       [0.0063093 , 0.0098516 , 0.0433223 , ..., 0.00964546, 0.00335754,
        0.00421889],
       [0.0432166 , 0.00399892, 0.00158078, ..., 0.01072805, 0.02863514,
        0.01042397],
       [0.04911343, 0.02557771, 0.01572509, ..., 0.03253119, 0.00780603,
        0.05665865]])

In [55]:
test_prob=pd.DataFrame(y_test_preds2)
test_prob.columns=["class_prob_%s"%i for i in range(1,20)]
test_prob["id"]=list(test_id["id"])
test_prob.to_csv('../pro/prob_bagging.csv',index=None)

In [56]:
test_prob

,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,class_prob_10,class_prob_11,class_prob_12,class_prob_13,class_prob_14,class_prob_15,class_prob_16,class_prob_17,class_prob_18,class_prob_19,id
0,0.043037,0.011172,0.017042,0.051765,0.996012,0.026449,0.002372,0.001829,0.000427,0.038009,0.064712,0.082411,0.002368,0.010772,0.003893,0.000308,0.004969,0.008380,0.002254,0
1,0.065482,0.001478,0.003387,0.997935,0.001667,0.005571,0.001223,0.001587,0.000319,0.015123,0.009185,0.042885,0.004156,0.000295,0.000694,0.000593,0.000756,0.002243,0.006742,1
2,0.386158,0.031399,0.743624,0.130455,0.159000,0.064726,0.183125,0.578888,0.094812,0.186884,0.552273,0.524910,0.951337,0.042581,0.102736,0.001065,0.050141,0.052403,0.134167,2
3,0.011062,0.001072,0.001093,0.998824,0.001892,0.011123,0.000731,0.000823,0.000317,0.008650,0.003348,0.001962,0.001741,0.000446,0.000581,0.000248,0.001094,0.000898,0.002047,3
4,0.043805,0.016331,0.047437,0.129443,0.994246,0.011494,0.287935,0.003892,0.002662,0.117143,0.035052,0.012867,0.016329,0.093231,0.188801,0.000303,0.006281,0.006807,0.010394,4
5,0.562552,0.060902,0.203231,0.010187,0.993896,0.087693,0.004048,0.091651,0.011534,0.201312,0.415374,0.190652,0.021564,0.016749,0.027012,0.001523,0.002424,0.067318,0.025660,5
6,0.158500,0.013648,0.060306,0.060035,0.000833,0.067807,0.008922,0.025138,0.000896,0.492128,0.083347,0.134902,0.011223,0.043701,0.972376,0.001230,0.075255,0.211174,0.223666,6
7,0.278018,0.930272,0.109379,0.044373,0.047481,0.029978,0.840354,0.116308,0.041682,0.479828,0.137111,0.282603,0.391734,0.073898,0.213497,0.000259,0.818850,0.190895,0.783242,7
8,0.001063,0.000324,0.998962,0.000769,0.001922,0.002655,0.000505,0.000759,0.000374,0.000340,0.001830,0.010370,0.000278,0.000328,0.000962,0.000542,0.000360,0.000601,0.001227,8
9,0.473034,0.072376,0.010996,0.030563,0.098622,0.042060,0.296368,0.778471,0.055713,0.827312,0.166199,0.943990,0.385488,0.007918,0.007783,0.001103,0.012632,0.090621,0.325857,9
